In [0]:
from google.colab import drive
drive.mount('/content/drive') 

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from pandas import datetime
import math, time
import itertools
from sklearn import preprocessing
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.layers import CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler

In [0]:
def get_scores(filename):
  print(filename)
  lstm = load_model(filename)
  predicted_stock_price = lstm.predict(X_test)
  predicted_stock_price.shape
  predicted_stock_price = np.hstack((predicted_stock_price, np.zeros((predicted_stock_price.shape[0], 1), dtype=predicted_stock_price.dtype))) # depending on the number of features.
  predicted_stock_price = sc.inverse_transform(predicted_stock_price) # denormalize values
  trainScore = lstm.evaluate(X_train1, y_train1, verbose=0)
  print('Train Score: %.20f MSE (%.20f RMSE)' % (trainScore, math.sqrt(trainScore)))
  testScore = lstm.evaluate(X_test, y_test, verbose=0)
  print('Test Score: %.20f MSE (%.20f RMSE)' % (testScore, math.sqrt(testScore)))
  rsp = real_stock_price.iloc[:,0:1].values
  predicted_stock_price = lstm.predict(X_test)
  predicted_stock_price = np.hstack((predicted_stock_price, np.zeros((predicted_stock_price.shape[0], 1), dtype=predicted_stock_price.dtype))) # depending on the number of features.
  testMSE = mean_squared_error(y_test_actual, sc.inverse_transform(predicted_stock_price)[:,0])
  testRMSE = np.sqrt(testMSE)
  print("Test MSE:",testMSE)
  print("Test RMSE:",testRMSE)

  predicted_stock_price = lstm.predict(X_train1)
  predicted_stock_price = np.hstack((predicted_stock_price, np.zeros((predicted_stock_price.shape[0], 1), dtype=predicted_stock_price.dtype))) # depending on the number of features.
  trainMSE = mean_squared_error(y_train_actual, sc.inverse_transform(predicted_stock_price)[:,0])
  trainRMSE = np.sqrt(trainMSE)
  print("Train MSE:",trainMSE)
  print("Train RMSE:",trainRMSE)

In [0]:
def getNormalizeXyTrain(training_set_scaled, amount_of_features, seq_len):
  #sentiment_coefficient_matrix = getExponentialDecayMatrix(seq_len)
  sentiment_coefficient_matrix = getSentimentMatrix(sentiment_coefficient,seq_len)
  X_train = []
  y_train = []
  for i in range(seq_len,1007):
      x_t = np.copy(training_set_scaled[i-seq_len:i,0:amount_of_features])
      X_train.append(x_t)
      y_train.append(training_set_scaled[i,0]) # predict the first row, open

  # applies to tweet sentiment
  for X in X_train:
    for index in range(len(sentiment_coefficient_matrix)):
        value = X[index:index+1,1]
        value = value*sentiment_coefficient_matrix[index]
        X[index:index+1,1] = value
        #X[index:index+1,1] = X[index:index+1,1] * sentiment_coefficient_matrix[index]

  X_train = np.asarray(X_train)
  y_train = np.asarray(y_train)
  print(X_train.shape) # no need reshape, already in 3d.
  print(y_train.shape)
  return X_train, y_train

In [0]:
def getSentimentMatrix(sentiment_coefficient,sequence_length):
  sentiment_coefficient_matrix = []
  # can explore e^ -time as the sentiment matrix
  for i in range(sequence_length):
    sentiment_coefficient_matrix.append(sentiment_coefficient**(sequence_length-(i+1)))
  return sentiment_coefficient_matrix


In [0]:
import math

def getExponentialDecayMatrix(sequence_length):
  exponential_decay_matrix = []
  for i in range(seq_len):
    exponential_decay_matrix.append(math.exp(-(sequence_length-(i+1))))
  return exponential_decay_matrix

In [0]:
num_hidden_layer = '1'
def build_LSTM(amount_of_features, seq_len, num_hidden_neuron, dropout_rate, ):
  seed_value = 8
  # Innitialise LSTM
  layers = [amount_of_features,seq_len,1]
  regressor = Sequential()

  #LSTM 1
  regressor.add(CuDNNLSTM(units=num_hidden_neuron, return_sequences=True, input_shape = (layers[1], layers[0]) ) )
  regressor.add(Dropout(dropout_rate, seed = seed_value)) 
  #LSTM 2
  regressor.add(CuDNNLSTM(units=num_hidden_neuron))
  regressor.add(Dropout(dropout_rate, seed = seed_value))
  
  # output layer
  opt = optimizers.Adam(lr=0.001, decay=1e-6)
  regressor.add(Dense(units=1, activation='relu'))
  regressor.compile(optimizer=opt, loss='mean_squared_error')

  return regressor

In [0]:
STOCK_NAME = 'spy'
TRAIN_FILE_NAME = STOCK_NAME + '_train.csv'
TEST_FILE_NAME =  STOCK_NAME + '_test.csv'
TRAIN_FILE = r'/content/drive/My Drive/stock_data/' + TRAIN_FILE_NAME
TEST_FILE = r'/content/drive/My Drive/stock_data/' + TEST_FILE_NAME
STORE_FOLDER = r'/content/drive/My Drive/' + STOCK_NAME + '_plot/CT/' 

path = '/content/drive/My Drive/' + STOCK_NAME + '_plot'
if not os.path.exists(path):
    os.makedirs(path)
path = '/content/drive/My Drive/' + STOCK_NAME + '_plot/CT'
if not os.path.exists(path):
    os.makedirs(path)

sentiment_coefficient = 0.075 # 0.001 (0.1%) 0.0001(0.01%) 0.00005(0.005%) 0.00001(0.001%)
seq_len = 3
amount_of_features = 2
num_of_epochs = 3000
size_of_batch = 32
p_val = 150
num_hid_units = 512
d_rate = 0.2
sentiment_coefficient_matrix = getSentimentMatrix(sentiment_coefficient,seq_len)

# Load train set
col_names = ['Date','Open','High','Low','Close','Adj Close','Volume','Tweet_Sentiment','Article_Sentiment']
dataset_train = pd.read_csv(TRAIN_FILE, header=0, names=col_names) 
ds_train = dataset_train
dataset_train = dataset_train.iloc[:1007,:].values # specify the number of rows
training_set = pd.DataFrame(dataset_train)
training_set.drop(training_set.columns[[0,1,2,3,5,6,8]],axis=1, inplace=True) # remove the columns u dont want
training_set.columns = ['Close','Tweet_Sentiment']
columnsTitles=['Close','Tweet_Sentiment'] # SWAP COLUMNS
training_set = training_set.reindex(columns=columnsTitles) # SWAP COLUMNS
#training_set.columns = ['Open','High','Low']

#Normalize data
sc = MinMaxScaler(feature_range=(0,1)) # feature range -> scale will be between 0 and 1
norm_close = sc.fit_transform(training_set['Close'].to_frame()) # normalize closing
norm_close = pd.DataFrame({'Close': norm_close[:, 0]})
frames = [norm_close, training_set['Tweet_Sentiment'].to_frame()]
training_set_scaled = pd.concat(frames, axis=1)
training_set_scaled = training_set_scaled.to_numpy()

# Load test set
col_names = ['Date','Open','High','Low','Close','Adj Close','Volume','Tweet_Sentiment','Article_Sentiment']
dataset_test = pd.read_csv(TEST_FILE, header=0, names=col_names)
ds_test = dataset_test
date = dataset_test.iloc[:251,0].values
dataset_test = dataset_test.iloc[:251,:].values

# realstockprice use for plotting later
real_stock_price = pd.DataFrame(dataset_test)
real_stock_price.drop(real_stock_price.columns[[0,1,2,3,5,6,8]],axis=1, inplace=True)
real_stock_price.columns = ['Close','Tweet_Sentiment']
real_stock_price = real_stock_price.reindex(columns=columnsTitles) # SWAP COLUMNS

# getting the sequence length based test data
dataset_total = pd.concat((ds_train[:1007], ds_test[:251]), axis = 0)
dataset_total.drop(dataset_total.columns[[0,1,2,3,5,6,8]],axis=1, inplace=True) ## drop here when remove feature
dataset_total = dataset_total.reindex(columns=columnsTitles) # SWAP COLUMNS
inputs = dataset_total[len(dataset_total)-len(ds_test) - seq_len:].values   # from the length of dataset test = 60 onwards

# for getting real MSE value
x_train_actual = []
y_train_actual = []
x_test_actual = [] 
y_test_actual = [] 
trainVals = training_set.values
for i in range(seq_len,1007):
    x_train_actual.append(trainVals[i-seq_len:i,0:amount_of_features])
    y_train_actual.append(trainVals[i,0]) # predict the first row, open

for i in range(seq_len, len(inputs)): # range depending on length of test data.
    y_test_actual.append(inputs[i,0]) 
    x_test_actual.append(inputs[i-seq_len:i,0:amount_of_features])

# normalize test inputs
norm_input_close = sc.transform(inputs[:,0:1])
norm_input_close = pd.DataFrame({'Close': norm_input_close[:,0]})
norm_input_tweet = pd.DataFrame({'Tweet_Sentiment': inputs[:,1]})
frames = [norm_input_close, norm_input_tweet]
inputs = pd.concat(frames, axis=1)
inputs = inputs.to_numpy()
X_test = []
y_test = []
for i in range(seq_len, len(inputs)): # range depending on length of test data.
    X_test.append(inputs[i-seq_len:i,0:amount_of_features])
    y_test.append(inputs[i,0]) 
X_test = np.array(X_test)
y_test = np.array(y_test)

In [0]:
feature = '_'
for i in range(len(real_stock_price.columns)):
  feature = feature + str(real_stock_price.columns[i])
file_name = (STORE_FOLDER +
            STOCK_NAME +
            '_e' + str(num_of_epochs) + 
            '_b' + str(size_of_batch) +
            '_f' + str(feature) +
            '_s' + str(seq_len) +
            '_h' + str(num_hid_units) +
            '_l' + str(num_hidden_layer) +
            '_sc' + str(sentiment_coefficient) +
            '_best_model.h5')
X_train1, y_train1 = getNormalizeXyTrain(training_set_scaled, amount_of_features, seq_len)
print(STOCK_NAME)
print(feature)
print(num_hid_units)
print(num_hidden_layer)
print(file_name)

In [0]:
exponential_flag = False # remember to change func used in getnormalize

lstm_list = []
testscore_list = []
testmse_list = []
testRMSE_list = []
finalTestScore_list = []
finalTestMSE_list = []
finalTestRMSE_list = []

for i in range(10): 
  if exponential_flag :
    file_name = (STORE_FOLDER +
                STOCK_NAME +
                '_e' + str(num_of_epochs) + 
                '_b' + str(size_of_batch) +
                '_f' + str(feature) +
                '_s' + str(seq_len) +
                '_h' + str(num_hid_units) +
                '_l' + str(num_hidden_layer) +
                '_scExponential'  +
                '_averageten_' +
                str(i) + 
                '_.h5')
  else:
    file_name = (STORE_FOLDER +
                STOCK_NAME +
                '_e' + str(num_of_epochs) + 
                '_b' + str(size_of_batch) +
                '_f' + str(feature) +
                '_s' + str(seq_len) +
                '_h' + str(num_hid_units) +
                '_l' + str(num_hidden_layer) +
                '_sc' + str(sentiment_coefficient) +
                '_averageten_' +
                str(i) + 
                '_.h5')
  print(file_name)

  if not os.path.exists(file_name):
    lstm = build_LSTM(amount_of_features, seq_len, num_hidden_neuron = num_hid_units, dropout_rate = d_rate)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = p_val)
    mc = ModelCheckpoint(file_name,  monitor='val_loss', mode='min', verbose=1, save_best_only=True)
    lstm.fit(X_train1, y_train1, validation_data = (X_test, y_test), epochs=num_of_epochs, batch_size=size_of_batch, callbacks=[es, mc], verbose=0)
  
  lstm_list.append(load_model(file_name))
  predicted_stock_price = lstm_list[i].predict(X_test)
  predicted_stock_price = sc.inverse_transform(predicted_stock_price) # denormalize values
  trainScore = lstm_list[i].evaluate(X_train1, y_train1, verbose=0)
  print('Train Score: %.20f MSE (%.20f RMSE)' % (trainScore, math.sqrt(trainScore)))
  testScore = lstm_list[i].evaluate(X_test, y_test, verbose=0)
  print('Test Score: %.20f MSE (%.20f RMSE)' % (testScore, math.sqrt(testScore)))
  rsp = real_stock_price.iloc[:,0:1].values
  predicted_stock_price = lstm_list[i].predict(X_test)
  testMSE = mean_squared_error(y_test_actual, sc.inverse_transform(predicted_stock_price)[:,0])
  testRMSE = np.sqrt(testMSE)
  print("Test MSE:",testMSE)
  print("Test RMSE:",testRMSE)

  predicted_stock_price = lstm_list[i].predict(X_train1)
  trainMSE = mean_squared_error(y_train_actual, sc.inverse_transform(predicted_stock_price)[:,0])
  trainRMSE = np.sqrt(trainMSE)
  print("Train MSE:",trainMSE)
  print("Train RMSE:",trainRMSE)

  testscore_list.append(testScore)
  testmse_list.append(testMSE)
  testRMSE_list.append(testRMSE)

t = 0
for i in testscore_list:
  t = t + i
print(t/10)
t = 0
for i in testmse_list:
  t = t + i
print(t/10)
t = 0
for i in testRMSE_list:
  t = t + i
print(t/10)
